In [ ]:
import spotipy
import matplotlib.pyplot as plt
import json
import pprint
import numpy as np
from datetime import datetime
from collections import defaultdict
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from datetime import datetime, timedelta
from sklearn import linear_model

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
import os
#print(os.environ)

with open('spotify_client.json', 'r') as json_file:
    client_info = json.load(json_file)
client_id = client_info[0]
client_secret = client_info[1]
auth_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=auth_manager)
#auth_manager=auth_manager

In [ ]:
json_2018_2020 = open("Streaming_History_Audio_2018-2020_0.json", encoding="utf8")
json_2020_2021 = open("Streaming_History_Audio_2020-2021_1.json", encoding="utf8")
json_2021_2022 = open("Streaming_History_Audio_2021-2022_2.json", encoding="utf8")
json_2022_2023 = open("Streaming_History_Audio_2022-2023_3.json", encoding="utf8")
json_2023 = open("Streaming_History_Audio_2023_4.json", encoding="utf8")
json_2023_2024 = open("Streaming_History_Audio_2023-2024_5.json",encoding="utf8")

hist_2018_2024 = [json.load(json_2018_2020)]
hist_2018_2024.append(json.load(json_2020_2021))
hist_2018_2024.append(json.load(json_2021_2022))
hist_2018_2024.append(json.load(json_2022_2023))
hist_2018_2024.append(json.load(json_2023))
hist_2018_2024.append(json.load(json_2023_2024))

json_2018_2020.close()
json_2020_2021.close()
json_2021_2022.close()
json_2022_2023.close()
json_2023.close()
json_2023_2024.close()

In [ ]:
uri_ts = []

for file in hist_2018_2024:
    for song in file:
        if song['reason_end'] == 'endplay' or song['reason_end'] == 'fwdbtn' or song['spotify_track_uri'] == None:
            continue
        uri = song['spotify_track_uri']
        ts = song['ts']
        uri_ts.append([uri, ts])

In [ ]:
features_ts = [[]]
batch_size = 99
# np.shape(uri_ts)[0]
for i in range(0, np.shape(uri_ts)[0], batch_size):
    features = np.array([sp.audio_features(np.array(uri_ts[i:i+batch_size])[:,0].tolist())])
    ts = [np.array(uri_ts[i:i+batch_size])[:,1]]
    
    features_ts_temp = features
    features_ts_temp = np.append(features_ts_temp, ts, axis=0)

    features_ts[i:i+batch_size] = features_ts_temp.T
    
data_to_serialize = [(row[0], row[1]) for row in features_ts]

with open('data.json', 'w') as json_file:
    json.dump(data_to_serialize, json_file, ensure_ascii=False, indent=4)

In [ ]:

with open('data.json', 'r') as json_file:
    retrieved_data = json.load(json_file)

retrieved_array = np.array(retrieved_data, dtype=object)


In [ ]:
valence_ts = []
for i in range(np.shape(retrieved_array)[0]):
    dict = retrieved_array[i,0]
    ts = retrieved_array[i,1]

    if dict is not None and 'valence' in dict:
        valence = dict['valence']
        date = datetime.strptime(ts, '%Y-%m-%dT%H:%M:%SZ')
        
        valence_ts.append([valence, date])
        
        
valence_ts = np.array(valence_ts)
print(np.shape(valence_ts))